In [43]:
## Import the dependencies
from geopy.geocoders import GoogleV3
from lat_lon_parser import parse
import pandas as pd, requests, time
import warnings
import numpy as np
import datetime as dt
import geopandas as gpd

## Import api key
from config import map_key

# 1. Setup

### 1.1 Import the Data

In [2]:
df = pd.read_csv (r'C:\Users\emich\Desktop\penn data science bootcamp\Project 3\Project-3-\Resources\Longer_Fireball_Data.csv')
df

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,2023-04-01 00:02:03,16.8S,76.0E,61.8,46.4,21.4,-40.6,7.1,2.400000e+10,0.086
1,2023-03-20 06:53:23,23.7S,132.6E,34.8,NaN,NaN,NaN,NaN,2.700000e+10,0.095
2,2023-03-11 08:21:28,74.7S,79.5E,32.0,NaN,NaN,NaN,NaN,4.440000e+11,1.100
3,2023-03-03 02:53:54,40.1N,87.4E,32.0,25.3,16.1,-19.3,2.6,6.400000e+10,0.200
4,2023-02-19 06:15:34,7.9N,109.9E,37.0,23.0,-1.1,-22.9,1.2,2.030000e+11,0.570
...,...,...,...,...,...,...,...,...,...,...
946,1993-11-29 17:48:41,26.5N,78.3E,NaN,NaN,NaN,NaN,NaN,2.600000e+10,0.092
947,1993-10-31 03:39:27,51.3N,100.9W,NaN,NaN,NaN,NaN,NaN,4.000000e+10,0.130
948,1991-10-04 09:22:47,78.7N,6.3E,NaN,NaN,NaN,NaN,NaN,5.500000e+11,1.400
949,1990-10-01 03:51:47,7.5N,142.8E,NaN,NaN,NaN,NaN,NaN,2.500000e+12,5.200


In [3]:
df.to_json(r'C:\Users\emich\Desktop\penn data science bootcamp\Project 3\Project-3-\Resources\Longer_Fireball_Data.json')

In [4]:
## Review the data type of each column
df.dtypes

Peak Brightness Date/Time (UT)          object
Latitude (deg.)                         object
Longitude (deg.)                        object
Altitude (km)                          float64
Velocity (km/s)                        float64
vx                                     float64
vy                                     float64
vz                                     float64
Total Radiated Energy (J)              float64
Calculated Total Impact Energy (kt)    float64
dtype: object

# 2. Data Cleaning

### 2.1 Remove the Unwanted Columns and Rows

In [5]:
## Drop all null values
fireball_df = df.dropna()
fireball_df

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,2023-04-01 00:02:03,16.8S,76.0E,61.8,46.4,21.4,-40.6,7.1,2.400000e+10,0.086
3,2023-03-03 02:53:54,40.1N,87.4E,32.0,25.3,16.1,-19.3,2.6,6.400000e+10,0.200
4,2023-02-19 06:15:34,7.9N,109.9E,37.0,23.0,-1.1,-22.9,1.2,2.030000e+11,0.570
5,2023-02-14 22:55:25,48.0S,53.3W,32.3,15.6,-15.1,3.7,-0.9,1.840000e+11,0.520
8,2023-01-17 06:56:59,33.1N,173.7W,38.0,14.2,7.8,-5.6,-10.5,2.900000e+10,0.100
...,...,...,...,...,...,...,...,...,...,...
680,2004-04-22 04:21:49,44.3S,83.3E,29.6,16.3,8.5,-12.1,-6.8,6.240000e+11,1.500
681,2004-04-06 03:24:49,12.0S,162.8E,35.0,25.6,10.5,-23.2,-2.9,4.330000e+11,1.100
692,2003-11-10 13:54:06,64.5S,136.2E,23.0,20.1,14.8,-8.7,10.4,5.180000e+11,1.300
697,2003-09-27 12:59:02,21.0N,86.6E,26.0,18.2,-1.0,-5.4,-17.3,2.150000e+12,4.600


In [6]:
## Drop 'vx', 'vy', 'vz' columns
fireball_df.pop('vx')
fireball_df

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,2023-04-01 00:02:03,16.8S,76.0E,61.8,46.4,-40.6,7.1,2.400000e+10,0.086
3,2023-03-03 02:53:54,40.1N,87.4E,32.0,25.3,-19.3,2.6,6.400000e+10,0.200
4,2023-02-19 06:15:34,7.9N,109.9E,37.0,23.0,-22.9,1.2,2.030000e+11,0.570
5,2023-02-14 22:55:25,48.0S,53.3W,32.3,15.6,3.7,-0.9,1.840000e+11,0.520
8,2023-01-17 06:56:59,33.1N,173.7W,38.0,14.2,-5.6,-10.5,2.900000e+10,0.100
...,...,...,...,...,...,...,...,...,...
680,2004-04-22 04:21:49,44.3S,83.3E,29.6,16.3,-12.1,-6.8,6.240000e+11,1.500
681,2004-04-06 03:24:49,12.0S,162.8E,35.0,25.6,-23.2,-2.9,4.330000e+11,1.100
692,2003-11-10 13:54:06,64.5S,136.2E,23.0,20.1,-8.7,10.4,5.180000e+11,1.300
697,2003-09-27 12:59:02,21.0N,86.6E,26.0,18.2,-5.4,-17.3,2.150000e+12,4.600


In [7]:
## Drop 'vx', 'vy', 'vz' columns
fireball_df.pop('vy')
fireball_df

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,2023-04-01 00:02:03,16.8S,76.0E,61.8,46.4,7.1,2.400000e+10,0.086
3,2023-03-03 02:53:54,40.1N,87.4E,32.0,25.3,2.6,6.400000e+10,0.200
4,2023-02-19 06:15:34,7.9N,109.9E,37.0,23.0,1.2,2.030000e+11,0.570
5,2023-02-14 22:55:25,48.0S,53.3W,32.3,15.6,-0.9,1.840000e+11,0.520
8,2023-01-17 06:56:59,33.1N,173.7W,38.0,14.2,-10.5,2.900000e+10,0.100
...,...,...,...,...,...,...,...,...
680,2004-04-22 04:21:49,44.3S,83.3E,29.6,16.3,-6.8,6.240000e+11,1.500
681,2004-04-06 03:24:49,12.0S,162.8E,35.0,25.6,-2.9,4.330000e+11,1.100
692,2003-11-10 13:54:06,64.5S,136.2E,23.0,20.1,10.4,5.180000e+11,1.300
697,2003-09-27 12:59:02,21.0N,86.6E,26.0,18.2,-17.3,2.150000e+12,4.600


In [8]:
## Drop 'vx', 'vy', 'vz' columns
fireball_df.pop('vz')
fireball_df

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,2023-04-01 00:02:03,16.8S,76.0E,61.8,46.4,2.400000e+10,0.086
3,2023-03-03 02:53:54,40.1N,87.4E,32.0,25.3,6.400000e+10,0.200
4,2023-02-19 06:15:34,7.9N,109.9E,37.0,23.0,2.030000e+11,0.570
5,2023-02-14 22:55:25,48.0S,53.3W,32.3,15.6,1.840000e+11,0.520
8,2023-01-17 06:56:59,33.1N,173.7W,38.0,14.2,2.900000e+10,0.100
...,...,...,...,...,...,...,...
680,2004-04-22 04:21:49,44.3S,83.3E,29.6,16.3,6.240000e+11,1.500
681,2004-04-06 03:24:49,12.0S,162.8E,35.0,25.6,4.330000e+11,1.100
692,2003-11-10 13:54:06,64.5S,136.2E,23.0,20.1,5.180000e+11,1.300
697,2003-09-27 12:59:02,21.0N,86.6E,26.0,18.2,2.150000e+12,4.600


In [9]:
# Split 'Peak Brightness Date/Time (UT)' into two columns: 'Peak Brightness Date' and 'Peak Brightness Time (UT)'
new_cols = fireball_df['Peak Brightness Date/Time (UT)'].str.split(expand=True)

# Rename the columns
new_cols.columns = ['Dates', 'Times (UT)']

# Concatenate the new DataFramewith the original data
fireball_df = pd.concat([fireball_df, new_cols], axis=1)

# Drop the orginal column 
fireball_df.drop('Peak Brightness Date/Time (UT)', axis=1, inplace=True)

# View the table
fireball_df

,Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt),Dates,Times (UT)
0,16.8S,76.0E,61.8,46.4,2.400000e+10,0.086,2023-04-01,00:02:03
3,40.1N,87.4E,32.0,25.3,6.400000e+10,0.200,2023-03-03,02:53:54
4,7.9N,109.9E,37.0,23.0,2.030000e+11,0.570,2023-02-19,06:15:34
5,48.0S,53.3W,32.3,15.6,1.840000e+11,0.520,2023-02-14,22:55:25
8,33.1N,173.7W,38.0,14.2,2.900000e+10,0.100,2023-01-17,06:56:59
...,...,...,...,...,...,...,...,...
680,44.3S,83.3E,29.6,16.3,6.240000e+11,1.500,2004-04-22,04:21:49
681,12.0S,162.8E,35.0,25.6,4.330000e+11,1.100,2004-04-06,03:24:49
692,64.5S,136.2E,23.0,20.1,5.180000e+11,1.300,2003-11-10,13:54:06
697,21.0N,86.6E,26.0,18.2,2.150000e+12,4.600,2003-09-27,12:59:02


In [10]:
# Split 'Peak Brightness Time (UT)' into 'hours', 'minutes', and 'seconds'
for index, row in fireball_df.iterrows():
    hour, minute, second = row['Times (UT)'].split(':')
    fireball_df.loc[index, 'hour'] = int(hour)
    fireball_df.loc[index, 'minute'] = int(minute)
    fireball_df.loc[index, 'second'] = int(second)
    
fireball_df

,Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt),Dates,Times (UT),hour,minute,second
0,16.8S,76.0E,61.8,46.4,2.400000e+10,0.086,2023-04-01,00:02:03,0.0,2.0,3.0
3,40.1N,87.4E,32.0,25.3,6.400000e+10,0.200,2023-03-03,02:53:54,2.0,53.0,54.0
4,7.9N,109.9E,37.0,23.0,2.030000e+11,0.570,2023-02-19,06:15:34,6.0,15.0,34.0
5,48.0S,53.3W,32.3,15.6,1.840000e+11,0.520,2023-02-14,22:55:25,22.0,55.0,25.0
8,33.1N,173.7W,38.0,14.2,2.900000e+10,0.100,2023-01-17,06:56:59,6.0,56.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...
680,44.3S,83.3E,29.6,16.3,6.240000e+11,1.500,2004-04-22,04:21:49,4.0,21.0,49.0
681,12.0S,162.8E,35.0,25.6,4.330000e+11,1.100,2004-04-06,03:24:49,3.0,24.0,49.0
692,64.5S,136.2E,23.0,20.1,5.180000e+11,1.300,2003-11-10,13:54:06,13.0,54.0,6.0
697,21.0N,86.6E,26.0,18.2,2.150000e+12,4.600,2003-09-27,12:59:02,12.0,59.0,2.0


In [11]:
# Convert all 'hours', 'minutes', 'seconds' into all minutes
fireball_df['total_minutes'] = fireball_df['hour'] * 60 + fireball_df['minute'] + fireball_df['second'] / 60

# Drop the orginal columns 'hours', 'minutes', 'seconds' 
fireball_df = fireball_df.drop(columns=['hour', 'minute', 'second'])

# View the table
fireball_df

,Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt),Dates,Times (UT),total_minutes
0,16.8S,76.0E,61.8,46.4,2.400000e+10,0.086,2023-04-01,00:02:03,2.050000
3,40.1N,87.4E,32.0,25.3,6.400000e+10,0.200,2023-03-03,02:53:54,173.900000
4,7.9N,109.9E,37.0,23.0,2.030000e+11,0.570,2023-02-19,06:15:34,375.566667
5,48.0S,53.3W,32.3,15.6,1.840000e+11,0.520,2023-02-14,22:55:25,1375.416667
8,33.1N,173.7W,38.0,14.2,2.900000e+10,0.100,2023-01-17,06:56:59,416.983333
...,...,...,...,...,...,...,...,...,...
680,44.3S,83.3E,29.6,16.3,6.240000e+11,1.500,2004-04-22,04:21:49,261.816667
681,12.0S,162.8E,35.0,25.6,4.330000e+11,1.100,2004-04-06,03:24:49,204.816667
692,64.5S,136.2E,23.0,20.1,5.180000e+11,1.300,2003-11-10,13:54:06,834.100000
697,21.0N,86.6E,26.0,18.2,2.150000e+12,4.600,2003-09-27,12:59:02,779.033333


In [12]:
for index, row in fireball_df.iterrows():
    try:
        hour, minute, second = row['Times (UT)'].split(':')
        df.loc[index, 'hour'] = int(hour)
        df.loc[index, 'minute'] = int(minute)
        df.loc[index, 'second'] = int(second)
    except:
        print(f"Error processing row {index}")

fireball_df

,Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt),Dates,Times (UT),total_minutes
0,16.8S,76.0E,61.8,46.4,2.400000e+10,0.086,2023-04-01,00:02:03,2.050000
3,40.1N,87.4E,32.0,25.3,6.400000e+10,0.200,2023-03-03,02:53:54,173.900000
4,7.9N,109.9E,37.0,23.0,2.030000e+11,0.570,2023-02-19,06:15:34,375.566667
5,48.0S,53.3W,32.3,15.6,1.840000e+11,0.520,2023-02-14,22:55:25,1375.416667
8,33.1N,173.7W,38.0,14.2,2.900000e+10,0.100,2023-01-17,06:56:59,416.983333
...,...,...,...,...,...,...,...,...,...
680,44.3S,83.3E,29.6,16.3,6.240000e+11,1.500,2004-04-22,04:21:49,261.816667
681,12.0S,162.8E,35.0,25.6,4.330000e+11,1.100,2004-04-06,03:24:49,204.816667
692,64.5S,136.2E,23.0,20.1,5.180000e+11,1.300,2003-11-10,13:54:06,834.100000
697,21.0N,86.6E,26.0,18.2,2.150000e+12,4.600,2003-09-27,12:59:02,779.033333


# 3. Latitude and Longitude Cleaning

In [13]:
## Because the dataset provides us with latitude and longitude for fireballs in degrees, we have to change it into coordinates. 
## Refer to this website: https://lweb.cfa.harvard.edu/space_geodesy/ATLAS/cme_convert.html when calculating coordinates properly
test = df.iloc[0]
test

Peak Brightness Date/Time (UT)         2023-04-01 00:02:03
Latitude (deg.)                                      16.8S
Longitude (deg.)                                     76.0E
Altitude (km)                                         61.8
Velocity (km/s)                                       46.4
vx                                                    21.4
vy                                                   -40.6
vz                                                     7.1
Total Radiated Energy (J)                    24000000000.0
Calculated Total Impact Energy (kt)                  0.086
hour                                                   0.0
minute                                                 2.0
second                                                 3.0
Name: 0, dtype: object

In [14]:
# Parse through the first row of "Latitude (deg.)" data
parse(test["Latitude (deg.)"])

-16.8

In [15]:
# Parse through the first row of "Longitude (deg.)" data
parse(test["Longitude (deg.)"])

76.0

In [16]:
# Parse through and clean all of 'fireball_lat' column
fireball_df2= fireball_df.loc[~(df["Latitude (deg.)"].isna())]
fireball_df2

,Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt),Dates,Times (UT),total_minutes
0,16.8S,76.0E,61.8,46.4,2.400000e+10,0.086,2023-04-01,00:02:03,2.050000
3,40.1N,87.4E,32.0,25.3,6.400000e+10,0.200,2023-03-03,02:53:54,173.900000
4,7.9N,109.9E,37.0,23.0,2.030000e+11,0.570,2023-02-19,06:15:34,375.566667
5,48.0S,53.3W,32.3,15.6,1.840000e+11,0.520,2023-02-14,22:55:25,1375.416667
8,33.1N,173.7W,38.0,14.2,2.900000e+10,0.100,2023-01-17,06:56:59,416.983333
...,...,...,...,...,...,...,...,...,...
680,44.3S,83.3E,29.6,16.3,6.240000e+11,1.500,2004-04-22,04:21:49,261.816667
681,12.0S,162.8E,35.0,25.6,4.330000e+11,1.100,2004-04-06,03:24:49,204.816667
692,64.5S,136.2E,23.0,20.1,5.180000e+11,1.300,2003-11-10,13:54:06,834.100000
697,21.0N,86.6E,26.0,18.2,2.150000e+12,4.600,2003-09-27,12:59:02,779.033333


In [17]:
# Create a new column that has the cleaned latitude coordinates and display it
fireball_df2["lat"] = fireball_df2["Latitude (deg.)"].map(lambda l: parse(l))

# Drop the orginal column 'Latitude (deg.)'
fireball_df2 = fireball_df2.drop(columns=['Latitude (deg.)'])
fireball_df2

,Longitude (deg.),Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt),Dates,Times (UT),total_minutes,lat
0,76.0E,61.8,46.4,2.400000e+10,0.086,2023-04-01,00:02:03,2.050000,-16.8
3,87.4E,32.0,25.3,6.400000e+10,0.200,2023-03-03,02:53:54,173.900000,40.1
4,109.9E,37.0,23.0,2.030000e+11,0.570,2023-02-19,06:15:34,375.566667,7.9
5,53.3W,32.3,15.6,1.840000e+11,0.520,2023-02-14,22:55:25,1375.416667,-48.0
8,173.7W,38.0,14.2,2.900000e+10,0.100,2023-01-17,06:56:59,416.983333,33.1
...,...,...,...,...,...,...,...,...,...
680,83.3E,29.6,16.3,6.240000e+11,1.500,2004-04-22,04:21:49,261.816667,-44.3
681,162.8E,35.0,25.6,4.330000e+11,1.100,2004-04-06,03:24:49,204.816667,-12.0
692,136.2E,23.0,20.1,5.180000e+11,1.300,2003-11-10,13:54:06,834.100000,-64.5
697,86.6E,26.0,18.2,2.150000e+12,4.600,2003-09-27,12:59:02,779.033333,21.0


In [18]:
# # Do the math (multiply the minutes with the latitude) to get coordinates for latitude
# ##fireball_df2['latitude'] = fireball_df2['total_minutes'] * fireball_df2['lat']
fireball_df2['latitude'] = fireball_df2['lat']
# # Drop 'lat' column
# fireball_df2 = fireball_df2.drop(columns=['lat'])
# fireball_df2

In [19]:
# Create a new column containing cleaned longitude coordinates and display it
fireball_df2["long"] = fireball_df2["Longitude (deg.)"].map(lambda l: parse(l))

# Drop the orginal column 'Longitude (deg.)'
fireball_df2 = fireball_df2.drop(columns=['Longitude (deg.)'])
fireball_df2

,Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt),Dates,Times (UT),total_minutes,lat,latitude,long
0,61.8,46.4,2.400000e+10,0.086,2023-04-01,00:02:03,2.050000,-16.8,-16.8,76.0
3,32.0,25.3,6.400000e+10,0.200,2023-03-03,02:53:54,173.900000,40.1,40.1,87.4
4,37.0,23.0,2.030000e+11,0.570,2023-02-19,06:15:34,375.566667,7.9,7.9,109.9
5,32.3,15.6,1.840000e+11,0.520,2023-02-14,22:55:25,1375.416667,-48.0,-48.0,-53.3
8,38.0,14.2,2.900000e+10,0.100,2023-01-17,06:56:59,416.983333,33.1,33.1,-173.7
...,...,...,...,...,...,...,...,...,...,...
680,29.6,16.3,6.240000e+11,1.500,2004-04-22,04:21:49,261.816667,-44.3,-44.3,83.3
681,35.0,25.6,4.330000e+11,1.100,2004-04-06,03:24:49,204.816667,-12.0,-12.0,162.8
692,23.0,20.1,5.180000e+11,1.300,2003-11-10,13:54:06,834.100000,-64.5,-64.5,136.2
697,26.0,18.2,2.150000e+12,4.600,2003-09-27,12:59:02,779.033333,21.0,21.0,86.6


In [20]:
# # Do the math (multiply the minutes with the longitude) to get coordinates for longitude
# fireball_df2['longitude'] = fireball_df2['total_minutes'] * fireball_df2['long']
fireball_df2['longitude'] = fireball_df2['long']
# # Drop 'lat' column
# fireball_df2 = fireball_df2.drop(columns=['long'])
# fireball_df2

In [21]:
# Add another column combining the latitude and longitude
fireball_df2 = fireball_df2.assign(geolocation=[*zip(fireball_df2.latitude, fireball_df2.longitude)])
fireball_df2

,Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt),Dates,Times (UT),total_minutes,lat,latitude,long,longitude,geolocation
0,61.8,46.4,2.400000e+10,0.086,2023-04-01,00:02:03,2.050000,-16.8,-16.8,76.0,76.0,"(-16.8, 76.0)"
3,32.0,25.3,6.400000e+10,0.200,2023-03-03,02:53:54,173.900000,40.1,40.1,87.4,87.4,"(40.1, 87.4)"
4,37.0,23.0,2.030000e+11,0.570,2023-02-19,06:15:34,375.566667,7.9,7.9,109.9,109.9,"(7.9, 109.9)"
5,32.3,15.6,1.840000e+11,0.520,2023-02-14,22:55:25,1375.416667,-48.0,-48.0,-53.3,-53.3,"(-48.0, -53.3)"
8,38.0,14.2,2.900000e+10,0.100,2023-01-17,06:56:59,416.983333,33.1,33.1,-173.7,-173.7,"(33.1, -173.7)"
...,...,...,...,...,...,...,...,...,...,...,...,...
680,29.6,16.3,6.240000e+11,1.500,2004-04-22,04:21:49,261.816667,-44.3,-44.3,83.3,83.3,"(-44.3, 83.3)"
681,35.0,25.6,4.330000e+11,1.100,2004-04-06,03:24:49,204.816667,-12.0,-12.0,162.8,162.8,"(-12.0, 162.8)"
692,23.0,20.1,5.180000e+11,1.300,2003-11-10,13:54:06,834.100000,-64.5,-64.5,136.2,136.2,"(-64.5, 136.2)"
697,26.0,18.2,2.150000e+12,4.600,2003-09-27,12:59:02,779.033333,21.0,21.0,86.6,86.6,"(21.0, 86.6)"


In [22]:
# Convert 'geolocation' to a string
fireball_df2['geolocation'] = fireball_df2['geolocation'].astype(str)

In [23]:
print(fireball_df2.dtypes)

Altitude (km)                          float64
Velocity (km/s)                        float64
Total Radiated Energy (J)              float64
Calculated Total Impact Energy (kt)    float64
Dates                                   object
Times (UT)                              object
total_minutes                          float64
lat                                    float64
latitude                               float64
long                                   float64
longitude                              float64
geolocation                             object
dtype: object


### 2.2 Filter out Fireballs that Fell Outside USA

In [24]:
## We are only looking at fireballs that fell or were reported within the USA, therefore we need to figure out what countries that fell over utilizing GeoLocation as the key.

fireball_df2.geolocation = (
    fireball_df2
    .geolocation
    .str
    .strip()
    .replace(
        {
            '\(':'',
            '\)':'',
            ' ':''
        },
        regex=True
    )
)
fireball_df2

,Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt),Dates,Times (UT),total_minutes,lat,latitude,long,longitude,geolocation
0,61.8,46.4,2.400000e+10,0.086,2023-04-01,00:02:03,2.050000,-16.8,-16.8,76.0,76.0,"-16.8,76.0"
3,32.0,25.3,6.400000e+10,0.200,2023-03-03,02:53:54,173.900000,40.1,40.1,87.4,87.4,"40.1,87.4"
4,37.0,23.0,2.030000e+11,0.570,2023-02-19,06:15:34,375.566667,7.9,7.9,109.9,109.9,"7.9,109.9"
5,32.3,15.6,1.840000e+11,0.520,2023-02-14,22:55:25,1375.416667,-48.0,-48.0,-53.3,-53.3,"-48.0,-53.3"
8,38.0,14.2,2.900000e+10,0.100,2023-01-17,06:56:59,416.983333,33.1,33.1,-173.7,-173.7,"33.1,-173.7"
...,...,...,...,...,...,...,...,...,...,...,...,...
680,29.6,16.3,6.240000e+11,1.500,2004-04-22,04:21:49,261.816667,-44.3,-44.3,83.3,83.3,"-44.3,83.3"
681,35.0,25.6,4.330000e+11,1.100,2004-04-06,03:24:49,204.816667,-12.0,-12.0,162.8,162.8,"-12.0,162.8"
692,23.0,20.1,5.180000e+11,1.300,2003-11-10,13:54:06,834.100000,-64.5,-64.5,136.2,136.2,"-64.5,136.2"
697,26.0,18.2,2.150000e+12,4.600,2003-09-27,12:59:02,779.033333,21.0,21.0,86.6,86.6,"21.0,86.6"


In [25]:
# Next, we want to narrow down the key to only unique locations
geo_key = pd.DataFrame(fireball_df2['geolocation'].unique(),columns=['geolocation'])
geo_key

,geolocation
0,"-16.8,76.0"
1,"40.1,87.4"
2,"7.9,109.9"
3,"-48.0,-53.3"
4,"33.1,-173.7"
...,...
280,"-44.3,83.3"
281,"-12.0,162.8"
282,"-64.5,136.2"
283,"21.0,86.6"


In [26]:
# Set the pause duration between api requests
pause = 0.1

In [27]:
# Set up a function that accepts a latitude,longitude string, sends it to the Google API, and returns the location API result
def geocode(latlong):
    time.sleep(pause) #pause for some duration before each request, to not hammer their server
    url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={}&sensor=false&key='+ map_key #api url with placeholders
    request = url.format(latlong) #fill in the placeholder with a variable
    response = requests.get(request) #send the request to the server and get the response
    data = response.json() #convert the response json string into a dict
    
    if len(data['results']) > 0: #if google was able to geolocate our address, extract formatted address from result
        return data['results'][0]['formatted_address']

In [28]:
geo_key.iloc[0:3,0].map(geocode)

0    5JMR6222+22
1    8MG939XX+XX
2    6PVFWV2X+2X
Name: geolocation, dtype: object

In [29]:
# for each value in the GeoLocation column, geocode it, save results as new df column
geo_key['Address'] = geo_key['geolocation'].map(geocode)

In [31]:
#Drop any incomplete sets
geo_key=geo_key.dropna()

# Check how it looks
geo_key.head()

,geolocation,Address
0,"-16.8,76.0",5JMR6222+22
1,"40.1,87.4",8MG939XX+XX
2,"7.9,109.9",6PVFWV2X+2X
3,"-48.0,-53.3",4838XPX2+X2
5,"31.5,18.6",8F3WGH2X+2X


In [32]:
# Filter out all non-USA results
country = 'USA'
geo_key2=geo_key[geo_key['Address'].str.contains(country)]
geo_key2['Country']='USA'
geo_key2.head()

C:\Users\emich\AppData\Local\Temp\ipykernel_28768\2735779226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_key2['Country']='USA'


,geolocation,Address,Country
230,"32.0,-92.9","2422+22 Goldonna, LA, USA",USA


In [33]:
#Merge the key back in with the dataset using an inner join to filter out non-USA meteorites
df3 = fireball_df2.merge(geo_key2, on='geolocation', how='inner')

#Drop columns not needed from key 
df3 = (df3.drop(['geolocation', 'Country'], axis=1).dropna())
df3.head()

,Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt),Dates,Times (UT),total_minutes,lat,latitude,long,longitude,Address
0,52.0,24.5,3.230000e+11,0.85,2010-03-08,22:02:07,1322.116667,32.0,32.0,-92.9,-92.9,"2422+22 Goldonna, LA, USA"


In [34]:
geo_key['Address'].unique()

array(['5JMR6222+22', '8MG939XX+XX', '6PVFWV2X+2X', '4838XPX2+X2',
       '8F3WGH2X+2X', '4Q6Q8222+22', '8MJJ8V2X+2X', '8C4VQW22+22',
       '757G2822+22', 'RC22+22 Dzhankeldy, Uzbekistan', '7VG72V2X+2X',
       '666M2422+22', '4C7V632X+2X', '4JFRGJ22+22', '598XPG22+22',
       '48922C22+22', '7QHG74X2+X2', '3V267MXX+XX', '84GJRV2X+2X',
       '37PHG422+22', '6H3VQQXX+XX', 'QPX2+X2 Abico, Coari - AM, Brazil',
       '6J4P78X2+X2', '69HXWC22+22', 'CC2G2W22+22', '6HQRCC22+22',
       '5F3H7CX2+X2', '5RR4P52X+2X', '99242222+22', '32HXHRX2+X2',
       '3P2Q72X2+X2', '9JJ2M8X2+X2',
       'HGX2+X2 Biyang County, Zhumadian, Henan, China',
       '6422+22 Sabalana, Pangkajene and Islands Regency, South Sulawesi, Indonesia',
       'GC22+22 Teploe, Orenburg Oblast, Russia', '9RFP272X+2X',
       '935JW222+22', '659CW52X+2X', '8MJWC92X+2X', '82PQ8R22+22',
       '4RQ38222+22', '77R2QWX2+X2', 'CP385MXX+XX', '4M3GC92X+2X',
       '2RC3W422+22', 'C679R622+22',
       '75XX+XX Labuan, Labuan Federa

In [41]:
#Save as JSON
df3.to_json('C:/Users/emich/Desktop/penn data science bootcamp/Project 3')

PermissionError: [Errno 13] Permission denied: 'C:/Users/emich/Desktop/penn data science bootcamp/Project 3'

In [44]:
# Set the Coordinate Reference System (CRS) to the WGS84 (epsg:4326) latitude-longitude projection 
# fyi: CRS tells Python how coordinates relate to places on the Earth
#      https://spatialreference.org/ref/epsg/4326/
#      https://en.wikipedia.org/wiki/World_Geodetic_System
crs={'init':'epsg:4326'}

# Create a GeoDataFrame using the zip code coordinates
geo_df=gpd.GeoDataFrame(df3,crs=crs,geometry=gpd.points_from_xy(df3['longitude'], df3['latitude']))

# Check dataframe
geo_df

C:\Users\emich\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Altitude (km),Velocity (km/s),Total Radiated Energy (J),Calculated Total Impact Energy (kt),Dates,Times (UT),total_minutes,lat,latitude,long,longitude,Address,geometry
0,52.0,24.5,3.230000e+11,0.85,2010-03-08,22:02:07,1322.116667,32.0,32.0,-92.9,-92.9,"2422+22 Goldonna, LA, USA",POINT (-92.90000 32.00000)


In [46]:
#Save as a GeoJSON
geo_df.to_file('Project-3-', driver="GeoJSON")  